In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!free -h

In [0]:
import os
print(os.getcwd())
os.chdir('/content/drive/My Drive/Google Colab/kaggle/PlantPathology')
print(os.getcwd())

In [0]:
# !unzip plant-pathology-2020-fgvc7.zip

In [0]:
import pandas as pd
from os.path import join
from glob import glob
from itertools import chain
from PIL import Image
from torchvision import datasets, models, transforms
import torch.utils.data as data

In [0]:
_data = pd.read_csv("train.csv")

healthy = (_data['healthy'][_data['healthy'] == 1])
multiple_diseases = (_data['multiple_diseases'][_data['multiple_diseases'] == 1])	
rust = (_data['rust'][_data['rust'] == 1])
scab = (_data['scab'][_data['scab'] == 1])

print("healthy:" , len(healthy))
print("multiple_diseases:" , len(multiple_diseases))
print("rust:" , len(rust))
print("scab:", len(scab))

TRAIN_HEALTHY_NUM = int(len(healthy) * 0.8)
print(TRAIN_HEALTHY_NUM)

TRAIN_MULTIPLE_NUM = int(len(multiple_diseases) * 0.8)
print(TRAIN_MULTIPLE_NUM)

TRAIN_RUST_NUM = int(len(rust) * 0.8)
print(TRAIN_RUST_NUM)

TRAIN_SCAB_NUM = int(len(scab) * 0.8)
print(TRAIN_SCAB_NUM)


In [0]:
class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(
                    resize, scale=(0.5, 1.0)),  # データオーギュメンテーション
                transforms.RandomHorizontalFlip(),  # データオーギュメンテーション
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ])
        }

    def __call__(self, img, phase='train'):
        return self.data_transform[phase](img)


In [0]:
def make_datapath_list(phase, class_data):
    TRAIN_MAX = {'healthy': TRAIN_HEALTHY_NUM, 'multiple_diseases': TRAIN_MULTIPLE_NUM, 'rust': TRAIN_RUST_NUM, 'scab': TRAIN_SCAB_NUM}
    train_list = []
    val_list = []
    class_names = ['healthy', 'multiple_diseases', 'rust', 'scab']
    count = {'healthy': 0, 'multiple_diseases': 0, 'rust': 0, 'scab': 0}

    for i in range(0, 1821):
        path = join("images", class_data.iloc[i, 0] + '.jpg')

        for j in range(1, 5):
            if class_data.iloc[i, j] == 1:
                if TRAIN_MAX[class_names[j - 1]] >= count[class_names[j - 1]]:
                    train_list.append(path)
                else:
                    val_list.append(path)
                
                count[class_names[j - 1]] += 1
                break

    
    return train_list, val_list


In [0]:
class LoadDataset(data.Dataset):
    
    def __init__(self, file_list, transform=None, phase='train', class_data=None):
        self.file_list = file_list  # ファイルパスのリスト
        self.transform = transform  # 前処理クラスのインスタンス
        self.phase = phase  # train or valの指定
        self.class_data = class_data
        
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        # index番目の画像をロード
        img_path = self.file_list[index]
        img = Image.open(img_path).convert("RGB")  # [高さ][幅][色RGB]

        # 画像の前処理を実施
        img_transformed = self.transform(
            img, self.phase)  # torch.Size([3, 224, 224])
        
        row = int(img_path[13:-4])
        _class_data = self.class_data.iloc[row]
        for i in range(1, 5):
            if _class_data[i] == 1:
                label = i - 1
        
        # 画像のラベルをファイル名から抜き出す
        return img_transformed, label

In [0]:
def make_dataLoader():
    
    size = 224
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    
    class_data = pd.read_csv("train.csv")

    batch_size = 32
    # trainとvalの画像へのパスを作成
    train_list, val_list = make_datapath_list(phase="val", class_data=class_data)

    # Datasetを作成する
    train_dataset = LoadDataset(
        file_list=train_list, transform=ImageTransform(size, mean, std), phase='train', class_data=class_data)

    val_dataset = LoadDataset(
        file_list=val_list, transform=ImageTransform(size, mean, std), phase='val', class_data=class_data)


    # DataLoaderを作成する
    train_dataloader = data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True)

    val_dataloader = data.DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False)

    # 辞書オブジェクトにまとめる
    dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

    return dataloaders_dict

In [0]:
 #モジュールのインポート
import random, yaml
import torch, torchvision
import numpy as np
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
from random import seed, sample
from sklearn.metrics import classification_report
from torchvision import datasets, models, transforms
from collections import OrderedDict
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
def fine_tuning_VGG16():
    # VGG-16モデルのインスタンスを生成
    use_pretrained = True  # 学習済みのパラメータを使用
    net = models.vgg16(pretrained=use_pretrained)

    net.classifier = nn.Sequential(OrderedDict([
        ("0", nn.Linear(25088, 4096)),
        ("1", nn.ReLU()),
        ("2", nn.Dropout(p=0.5)),
        ("3", nn.Linear(4096, 4))
    ]))


    # 訓練モードに設定
    net.train()
    print('ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました')

    #以下fine turning
    params_to_update_1 = []
    params_to_update_2 = []
    params_to_update_3 = []

    # 学習させる層のパラメータ名を指定
    update_param_names_1 = ["features.21.weight", "features.21.bias","features.24.weight", "features.24.bias","features.26.weight", "features.26.bias", "features.28.weight", "features.28.bias"]
    update_param_names_2 = ["classifier.0.weight",
                            "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
    # update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

    # パラメータごとに各リストに格納する
    for name, param in net.named_parameters():
        if name in update_param_names_1:
            param.requires_grad = True
            params_to_update_1.append(param)
            print("params_to_update_1に格納：", name)

        elif name in update_param_names_2:
            param.requires_grad = True
            params_to_update_2.append(param)
            print("params_to_update_2に格納：", name)

        # elif name in update_param_names_3:
        #     param.requires_grad = True
        #     params_to_update_3.append(param)
        #     print("params_to_update_3に格納：", name)

        else:
            param.requires_grad = False
            print("勾配計算なし。学習しない：", name)
        
        print('fine_tuning設定完了')
        # Optimizer設定
    optimizer = optim.SGD([
        {'params': params_to_update_1, 'lr': 5e-4},
        {'params': params_to_update_2, 'lr': 5e-3},
        # {'params': params_to_update_3, 'lr': 5e-3}
    ], momentum=0.9)

    return net,optimizer

In [0]:
def do_train_return_dic(net, dataloader_dict, criterion, optimizer, num_epochs):

    # 初期設定
    # GPUが使えるかを確認
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net = net.to(device)
    if device == 'cuda':
        net = torch.nn.DataParallel(net) # make parallel

        # ネットワークがある程度固定であれば、高速化させる
        torch.backends.cudnn.benchmark = True


    epoch_loss_dic = {}
    epoch_acc_dic = {}

    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
            if (epoch == 0) and (phase == 'train'):
                continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloader_dict[phase]):

                # GPUが使えるならGPUにデータを送る
                inputs = inputs.to(device)
                labels = labels.to(device)

                # optimizerを初期化
                optimizer.zero_grad()


                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloader_dict[phase].dataset)

            epoch_loss_dic.setdefault(phase, []).append(epoch_loss)
            epoch_acc_dic.setdefault(phase, []).append(epoch_acc.to("cpu").item())

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
    # PyTorchのネットワークパラメータの保存
    save_path = './weights_fine_tuning_drill.pth'
    torch.save(net.state_dict(), save_path)
    
    return epoch_loss_dic, epoch_acc_dic

In [0]:
def do_test(dataloaders_dict):
    net = models.vgg16(False)

    net.classifier = nn.Sequential(OrderedDict([
        ("0", nn.Linear(25088, 4096)),
        ("1", nn.ReLU()),
        ("2", nn.Dropout(p=0.5)),
        ("3", nn.Linear(4096, 4))
    ]))

    net_weights = torch.load('./weights_fine_tuning_drill.pth', map_location={'cuda': 'cpu'})
    new_state_dict = OrderedDict()

    for k, v in net_weights.items():
        k = k[7:]
        new_state_dict[k]=v

    net.load_state_dict(new_state_dict)

    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("使用デバイス：", device)

    net = net.to(device)
    if device == 'cuda':
        net = torch.nn.DataParallel(net) # make parallel

        # ネットワークがある程度固定であれば、高速化させる
        torch.backends.cudnn.benchmark = True

    net.eval()

    Y,pred = [], []
    for inputs, labels in tqdm(dataloaders_dict["val"]):

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = net(inputs)
        _, preds = torch.max(outputs, 1)  # ラベルを予測

        Y.extend(labels.to("cpu").numpy().tolist())
        pred.extend(preds.to("cpu").numpy().tolist())

    print(classification_report(Y, pred))

In [0]:
 def plot_history_loss(loss):
    # hist.historyに辞書型で損失値や精度が入っているので取得して表示
    plt.plot(loss['train'],label="loss for training")
    plt.plot(loss['val'],label="loss for validation")
    
    #matplotlibの細かい設定
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='best')
    
    plt.show()
    

def plot_history_acc(acc):
    plt.plot(acc['train'],label="acc for training")
    plt.plot(acc['val'],label="acc for validation")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc='best')
    plt.ylim([0, 1])
    plt.show()

In [0]:
def main():
    dataloaders_dict = make_dataLoader() #trとvalのデータ整形
    
    net,optimizer = fine_tuning_VGG16() # VGGF16モデルを読み込む
    criterion = nn.CrossEntropyLoss()  # 損失関数の設定
    
    # 学習・検証を実行する
    num_epochs = 40
    epoch_loss_dic, epoch_acc_dic = do_train_return_dic(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)
    
    plot_history_loss(epoch_loss_dic)

    plot_history_acc(epoch_acc_dic)

    # 検証
    # do_test(dataloaders_dict)



In [0]:
if __name__=='__main__':
    # 乱数のシードを設定
    torch.manual_seed(1234)
    np.random.seed(1234)
    random.seed(1234)
    
    main()

In [0]:
dataloaders_dict = make_dataLoader() #trとvalのデータ整形

do_test(dataloaders_dict)

In [0]:
test = pd.read_csv("test.csv")
test_path_list = [join('images', path + '.jpg') for path in chain.from_iterable(test.values.tolist())]

size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

test_transform = transforms.Compose([
                transforms.Resize(size),  # リサイズ
                transforms.CenterCrop(size),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ])
trained_net = models.vgg16(False)

trained_net.classifier = nn.Sequential(OrderedDict([
    ("0", nn.Linear(25088, 4096)),
    ("1", nn.ReLU()),
    ("2", nn.Dropout(p=0.5)),
    ("3", nn.Linear(4096, 4))
]))

net_weights = torch.load('./weights_fine_tuning_drill.pth', map_location={'cuda': 'cpu'})
from collections import OrderedDict
new_state_dict = OrderedDict()

for k, v in net_weights.items():
    k = k[7:]
    new_state_dict[k]=v

trained_net.load_state_dict(new_state_dict)


trained_net.eval()

preds = []
# print(test_path_list)
# img = Image.open(test_path_list[0]).convert("RGB")
# img_transformed = test_transform(img).unsqueeze(0)
# o = trained_net(img_transformed)
# pred = int(o.argmax())
# print(pred)

for path in tqdm(test_path_list):
    img = Image.open(path).convert("RGB")  # [高さ][幅][色RGB]

    img_transformed = test_transform(img).unsqueeze(0)  # torch.Size([3, 224, 224])
    
    output = trained_net(img_transformed)
    preds += [int(output.argmax())]

print('success')

In [0]:
print(len(preds))

In [0]:
id_list = [p[7:-4] for p in test_path_list]

pred_dict = {'healthy': [0] * 1821, 'multiple_diseases': [0] * 1821, 'rust': [0] * 1821, 'scab': [0] * 1821}
class_list = ['healthy', 'multiple_diseases', 'rust', 'scab']

for i, pred in enumerate(preds):
    pred_dict[class_list[pred]][i] = 1

submit_file = pd.DataFrame({'image_id':id_list, 'healthy': pred_dict['healthy'], 'multiple_diseases': pred_dict['multiple_diseases'], 'rust': pred_dict['rust'], 'scab': pred_dict['scab']})
submit_file.to_csv('submit.csv', index = False)